In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/10-monkey-species/monkey_labels.txt
/kaggle/input/10-monkey-species/validation/validation/n9/n9024.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n916.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n907.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n9030.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n918.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n902.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n914.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n912.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n9015.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n903.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n908.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n913.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n905.jpg
/kaggle/input/10-monkey-species/validation/validation/n9/n9013.jpg
/kaggle/input/10-monke

**Import Libraries and Datasets**

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization, MaxPooling2D

training_data = '../input/10-monkey-species/training/training' 
validation_data = '../input/10-monkey-species/validation/validation/'
labels_path = '../input/monkey_labels.txt'

print("Setup Done")

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Setup Done


/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

**Prep Data**

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224
num_classes = 10
bs = 10

def prep(image):
    img = np.array(image)
    img /= 255
    return img

aug_data_generator = ImageDataGenerator(preprocessing_function=prep,
                                    horizontal_flip = True,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1)
data_generator = ImageDataGenerator(preprocessing_function=prep)

train_generator = aug_data_generator.flow_from_directory(
                                    directory=training_data,
                                    target_size=(image_size, image_size),
                                    batch_size=bs,
                                    class_mode='categorical')

valid_generator = aug_data_generator.flow_from_directory(
                                    directory=validation_data,
                                    target_size=(image_size, image_size),
                                    class_mode='categorical')


Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


**Develop Model**

In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', strides=2,
                 input_shape=(image_size, image_size, 3),
                 padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid', data_format=None))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', strides=1, padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid', data_format=None))

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', strides=2, padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid', data_format=None))
model.add(Dropout(0.3))
model.add(Conv2D(32, (3, 3), activation='relu', strides=1, padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

**Compile Model**

In [5]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

**Fit Model**

In [6]:
model.fit_generator(train_generator, 
                    epochs=10,
                    steps_per_epoch=1370/bs,
                   validation_data=valid_generator,
                   validation_steps=1)

Epoch 1/10
137/137 [==============================] - 58s 424ms/step - loss: 2.2672 - acc: 0.1382 - val_loss: 2.2651 - val_acc: 0.1250
Epoch 2/10
137/137 [==============================] - 53s 386ms/step - loss: 1.8657 - acc: 0.3221 - val_loss: 1.7084 - val_acc: 0.3125
Epoch 3/10
137/137 [==============================] - 55s 402ms/step - loss: 1.5912 - acc: 0.4320 - val_loss: 1.3483 - val_acc: 0.4688
Epoch 4/10
137/137 [==============================] - 57s 417ms/step - loss: 1.3519 - acc: 0.5022 - val_loss: 1.2790 - val_acc: 0.5312
Epoch 5/10
137/137 [==============================] - 55s 404ms/step - loss: 1.3070 - acc: 0.5256 - val_loss: 1.4054 - val_acc: 0.5000
Epoch 6/10
137/137 [==============================] - 55s 402ms/step - loss: 1.1302 - acc: 0.5747 - val_loss: 1.2347 - val_acc: 0.5625
Epoch 7/10
137/137 [==============================] - 57s 418ms/step - loss: 1.0984 - acc: 0.6111 - val_loss: 1.1358 - val_acc: 0.6250
Epoch 8/10
137/137 [==============================] - 5

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 55, 55, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 55, 55, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0